# <통합 F1-score : 약 0.75, 임직원 : 약 0.79, 비임직원 : 약 0.70>

> train4 데이터는 어제 올려주신 ML0520에서 만들어진 train2.csv에 scd별로 groupby한 데이터셋에 추가적인 변수 3개 ('is_total_qty_than_9(특정 손님의 총 구매 수량이 9 이상인가)', 'is_price_higher_than_avg_amt_for_scd_cnt'(손님의 주문 건수별로 groupby하였을 때 주문당 평균적인 가격은 어느 정도이고, 이보다 높은 가격을 지불했는가), 'is_price_lower_than_avg(본 주문은 같은 상품에 대한 주문 데이터들의 평균 가격보다 낮은가)') 를 추가한 데이터입니다.

> 이 세개의 변수는 각 주문별 데이터에서 True, False 값으로 이루어져있으며, avg_amt_for_scd_cnt 랑 is_price_lower_than_avg는 각각의 주문별로 나타나는 boolean값이므로, mean()을 계산하여, 특정 손님이 주문하는 내역 중 특정치와 비교하여 가격이 높고 낮은 비율이 어느정도인지를 파악하도록 하였습니다.

In [ ]:
!pip install --upgrade pip
!pip install -U pypandoc
!pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.6/483.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparin

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import re
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/BDA 공모전/train4.csv', encoding='cp949') # 인코딩 방식은 각자 컴퓨터에 맞게 바꿔주세요!

In [ ]:
data

,scd,net_order_qty,net_order_amt,gender,age_grp,employee_yn,prime_yn,box,date,day_of_week,...,행복한콩,다담,크레잇,메티에,쁘띠첼,맥스봉,맛밤,is_price_lower_than_avg,is_total_qty_than_9,is_price_higher_than_avg_amt_for_scd_cnt
0,20230101963226,1,10.847569,M,3,Y,N,True,1,1,...,0,0,0,0,0,0,0,0.0,0,0.5
1,20230101963235,1,8.883224,F,4,Y,Y,False,1,1,...,0,0,0,0,0,0,0,1.0,0,0.5
2,20230101963244,3,10.234373,F,4,N,Y,False,1,1,...,0,0,0,0,0,0,0,0.0,0,0.0
3,20230101963247,1,9.055206,M,3,Y,Y,False,1,1,...,0,0,0,0,0,0,0,0.0,0,0.2
4,20230101963251,2,10.845621,F,3,Y,Y,False,1,1,...,0,0,0,0,0,0,0,0.0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10648,20230131216756,1,8.869820,F,4,N,N,False,0,0,...,0,0,0,0,0,0,0,0.0,0,0.0
10649,20230131216771,2,9.125762,F,4,N,Y,False,0,0,...,0,0,0,0,0,0,0,1.0,0,0.0
10650,20230131216842,1,10.176411,M,3,N,N,True,0,0,...,0,0,0,0,0,0,0,1.0,0,0.0
10651,20230131216844,1,8.609043,F,4,N,N,False,0,0,...,0,0,0,0,0,0,0,0.0,0,0.4


In [ ]:
data.loc[:,data.dtypes == bool] = data.loc[:,data.dtypes == bool].astype('int')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10653 entries, 0 to 10652
Data columns (total 37 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   scd                                       10653 non-null  int64  
 1   net_order_qty                             10653 non-null  int64  
 2   net_order_amt                             10653 non-null  float64
 3   gender                                    10653 non-null  object 
 4   age_grp                                   10653 non-null  int64  
 5   employee_yn                               10653 non-null  object 
 6   prime_yn                                  10653 non-null  object 
 7   box                                       10653 non-null  int64  
 8   date                                      10653 non-null  int64  
 9   day_of_week                               10653 non-null  int64  
 10  holiday                           

In [ ]:
from pycaret.classification import *

In [ ]:
data.columns

Index(['scd', 'net_order_qty', 'net_order_amt', 'gender', 'age_grp',
       'employee_yn', 'prime_yn', 'box', 'date', 'day_of_week', 'holiday',
       'bulk', 'sale', 'limit', '비비고', '햇반', '고메', '백설', '쿡킷', 'The 더건강한',
       '스팸', '다시다', '해찬들', '유산균', '한뿌리', '삼호어묵', '하선정', '행복한콩', '다담', '크레잇',
       '메티에', '쁘띠첼', '맥스봉', '맛밤', 'is_price_lower_than_avg',
       'is_total_qty_than_9', 'is_price_higher_than_avg_amt_for_scd_cnt'],
      dtype='object')

'is_price_lower_than_avg' 랑 'is_price_higher_than_avg_amt_for_scd_cnt'는 연속적으로 나타나는 수치형 변수이므로 numerical 변수로, 'is_total_qty_than_9'는 0 또는 1로만 나타나는 binary 변수이므로 categorical 변수로 추가하겠습니다.

추가적으로, age_grp도 원래는 categorical 변수로 보려고 하였으나, 가설2에서 연령이 증가할수록 (10대 < 20대 < 30대 < 40대 < 50대 < 60대) 순서대로 프라임 직원의 비율이 높게 나온 것을 확인하였으므로, numerical 변수로 분류해보겠습니다!

In [ ]:
cate = ['gender' ,'employee_yn', 'box', 'date', 'day_of_week', 'holiday',
       'bulk', 'sale', 'limit', '비비고', '햇반', '고메', '백설', '쿡킷', 'The 더건강한',
       '스팸', '다시다', '해찬들', '유산균', '한뿌리', '삼호어묵', '하선정', '행복한콩', '다담', '크레잇',
       '메티에', '쁘띠첼', '맥스봉', '맛밤',
       'is_total_qty_than_9']

# 임직원 유무 데이터

In [ ]:
employee = data[data['employee_yn'] == 'Y']
custom = data[data['employee_yn'] == 'N']

In [ ]:
data

,scd,net_order_qty,net_order_amt,gender,age_grp,employee_yn,prime_yn,box,date,day_of_week,...,행복한콩,다담,크레잇,메티에,쁘띠첼,맥스봉,맛밤,is_price_lower_than_avg,is_total_qty_than_9,is_price_higher_than_avg_amt_for_scd_cnt
0,20230101963226,1,10.847569,M,3,Y,N,1,1,1,...,0,0,0,0,0,0,0,0.0,0,0.5
1,20230101963235,1,8.883224,F,4,Y,Y,0,1,1,...,0,0,0,0,0,0,0,1.0,0,0.5
2,20230101963244,3,10.234373,F,4,N,Y,0,1,1,...,0,0,0,0,0,0,0,0.0,0,0.0
3,20230101963247,1,9.055206,M,3,Y,Y,0,1,1,...,0,0,0,0,0,0,0,0.0,0,0.2
4,20230101963251,2,10.845621,F,3,Y,Y,0,1,1,...,0,0,0,0,0,0,0,0.0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10648,20230131216756,1,8.869820,F,4,N,N,0,0,0,...,0,0,0,0,0,0,0,0.0,0,0.0
10649,20230131216771,2,9.125762,F,4,N,Y,0,0,0,...,0,0,0,0,0,0,0,1.0,0,0.0
10650,20230131216842,1,10.176411,M,3,N,N,1,0,0,...,0,0,0,0,0,0,0,1.0,0,0.0
10651,20230131216844,1,8.609043,F,4,N,N,0,0,0,...,0,0,0,0,0,0,0,0.0,0,0.4


훈련 데이터에 고객의 정보(scd)는 들어가지 않는 것이 좋을 것 같아서 뺐습니다! -> data = data.iloc[:, 1:]을 사용

In [ ]:
clf1 = setup(data = data.iloc[:, 1:],
             target = 'prime_yn',
             numeric_imputation = 'mean',
             categorical_features = cate
            )

,Description,Value
0,Session id,3211
1,Target,prime_yn
2,Target type,Binary
3,Target mapping,"N: 0, Y: 1"
4,Original data shape,"(10653, 36)"
5,Transformed data shape,"(10653, 199)"
6,Transformed train set shape,"(7457, 199)"
7,Transformed test set shape,"(3196, 199)"
8,Ordinal features,7
9,Numeric features,5


In [ ]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.7617,0.8454,0.7475,0.7691,0.7580,0.5234,0.5238,7.6900
lightgbm,Light Gradient Boosting Machine,0.7381,0.8273,0.7215,0.7461,0.7335,0.4762,0.4766,1.1480
gbc,Gradient Boosting Classifier,0.7188,0.8002,0.6888,0.7326,0.7099,0.4376,0.4385,3.4250
rf,Random Forest Classifier,0.7050,0.7795,0.7041,0.7051,0.7044,0.4100,0.4102,3.4830
ada,Ada Boost Classifier,0.7024,0.7676,0.7193,0.6961,0.7072,0.4049,0.4054,2.2310
lr,Logistic Regression,0.6949,0.7661,0.6984,0.6936,0.6959,0.3898,0.3900,4.2800
ridge,Ridge Classifier,0.6940,0.0000,0.6928,0.6944,0.6935,0.3880,0.3881,0.7440
lda,Linear Discriminant Analysis,0.6937,0.7615,0.6922,0.6942,0.6931,0.3874,0.3876,1.4250
dt,Decision Tree Classifier,0.6823,0.6822,0.6772,0.6841,0.6805,0.3646,0.3648,1.4940
et,Extra Trees Classifier,0.6795,0.7420,0.6762,0.6804,0.6782,0.3590,0.3592,3.4410


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=-1, num_parallel_tree=None,
              objective='binary:logistic', predictor=None, ...)

In [ ]:
xgboost = create_model('xgboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7735,0.8563,0.7560,0.7833,0.7694,0.5469,0.5472
1,0.7627,0.8450,0.7507,0.7692,0.7598,0.5255,0.5256
2,0.7373,0.8257,0.6997,0.7565,0.7270,0.4745,0.4759
3,0.7399,0.8199,0.7239,0.7479,0.7357,0.4799,0.4801
4,0.7453,0.8274,0.7265,0.7549,0.7404,0.4906,0.4910
5,0.7654,0.8550,0.7399,0.7797,0.7593,0.5308,0.5315
6,0.7895,0.8696,0.7962,0.7857,0.7909,0.5791,0.5791
7,0.7651,0.8513,0.7581,0.7684,0.7632,0.5302,0.5302
8,0.7758,0.8664,0.7849,0.7704,0.7776,0.5517,0.5518


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

# 임직원 예측

In [ ]:
employee = employee.drop('employee_yn', axis=1)

In [ ]:
cate_2 = ['gender', 'box', 'date', 'day_of_week', 'holiday',
       'bulk', 'sale', 'limit', '비비고', '햇반', '고메', '백설', '쿡킷', 'The 더건강한',
       '스팸', '다시다', '해찬들', '유산균', '한뿌리', '삼호어묵', '하선정', '행복한콩', '다담', '크레잇',
       '메티에', '쁘띠첼', '맥스봉', '맛밤',
       'is_total_qty_than_9']

In [ ]:
clf2 = setup(data = employee.iloc[:, 1:],
             target = 'prime_yn',
             numeric_imputation = 'mean',
             categorical_features = cate_2
            )

,Description,Value
0,Session id,6566
1,Target,prime_yn
2,Target type,Binary
3,Target mapping,"N: 0, Y: 1"
4,Original data shape,"(4713, 35)"
5,Transformed data shape,"(4713, 167)"
6,Transformed train set shape,"(3299, 167)"
7,Transformed test set shape,"(1414, 167)"
8,Ordinal features,6
9,Numeric features,5


In [ ]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.7533,0.8202,0.8046,0.7890,0.7964,0.4831,0.4839,3.4510
lightgbm,Light Gradient Boosting Machine,0.7351,0.8062,0.7905,0.7734,0.7813,0.4450,0.4461,1.1110
gbc,Gradient Boosting Classifier,0.7208,0.7804,0.7567,0.7735,0.7646,0.4215,0.4222,1.9270
rf,Random Forest Classifier,0.7033,0.7606,0.7890,0.7360,0.7614,0.3701,0.3720,1.8600
lr,Logistic Regression,0.7011,0.7526,0.7299,0.7627,0.7453,0.3837,0.3850,3.5630
ada,Ada Boost Classifier,0.6996,0.7481,0.7188,0.7668,0.7415,0.3836,0.3854,1.4410
ridge,Ridge Classifier,0.6954,0.0000,0.7133,0.7641,0.7371,0.3756,0.3776,0.5560
lda,Linear Discriminant Analysis,0.6944,0.7398,0.7097,0.7649,0.7356,0.3746,0.3769,1.7580
et,Extra Trees Classifier,0.6754,0.7208,0.7759,0.7102,0.7412,0.3075,0.3105,2.0020
dt,Decision Tree Classifier,0.6644,0.6479,0.7289,0.7171,0.7227,0.2978,0.2982,0.7190


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=-1, num_parallel_tree=None,
              objective='binary:logistic', predictor=None, ...)

# 비임직원 예측

In [ ]:
custom =  custom.drop('employee_yn', axis=1)

In [ ]:
clf3 = setup(data = custom.iloc[:, 1:],
             target = 'prime_yn',
             numeric_imputation = 'mean',
             categorical_features = cate_2
            )

,Description,Value
0,Session id,4532
1,Target,prime_yn
2,Target type,Binary
3,Target mapping,"N: 0, Y: 1"
4,Original data shape,"(5940, 35)"
5,Transformed data shape,"(5940, 171)"
6,Transformed train set shape,"(4158, 171)"
7,Transformed test set shape,"(1782, 171)"
8,Ordinal features,7
9,Numeric features,5


In [ ]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.7573,0.8434,0.6833,0.7251,0.7025,0.4981,0.4997,4.4120
lightgbm,Light Gradient Boosting Machine,0.7472,0.8257,0.6575,0.7178,0.6853,0.4750,0.4770,1.1240
gbc,Gradient Boosting Classifier,0.7253,0.8062,0.6060,0.7004,0.6489,0.4255,0.4291,2.1390
rf,Random Forest Classifier,0.7179,0.7901,0.5996,0.6881,0.6404,0.4103,0.4132,2.3780
et,Extra Trees Classifier,0.6991,0.7551,0.5888,0.6601,0.6216,0.3733,0.3756,2.1580
ada,Ada Boost Classifier,0.6962,0.7507,0.6215,0.6440,0.6315,0.3735,0.3745,1.5200
dt,Decision Tree Classifier,0.6953,0.6867,0.6329,0.6388,0.6350,0.3736,0.3743,0.8430
lr,Logistic Regression,0.6931,0.7496,0.6014,0.6441,0.6211,0.3640,0.3653,2.5640
ridge,Ridge Classifier,0.6895,0.0000,0.6077,0.6364,0.6209,0.3585,0.3594,0.9910
lda,Linear Discriminant Analysis,0.6876,0.7403,0.6054,0.6341,0.6187,0.3545,0.3554,1.1790


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=-1, num_parallel_tree=None,
              objective='binary:logistic', predictor=None, ...)